In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from functools import partial
import sys
sys.path.append('/home/tomek/ib_tools')
from indicators import get_ATR, get_signals
%matplotlib inline

In [2]:
contract = pd.read_pickle('data/minute_NQ_cont_non_active_included.pickle')
multiplier = 20
min_tick = 0.25

In [3]:
contract = contract.loc['20180202':]
contract

,open,high,low,close,volume,average,barCount
date,,,,,,,
2018-02-02 00:00:00,6954.00,6958.00,6953.50,6955.25,877,6955.025,362
2018-02-02 00:01:00,6955.25,6956.50,6952.50,6952.50,376,6955.150,201
2018-02-02 00:02:00,6952.50,6954.00,6951.25,6952.00,292,6952.525,141
2018-02-02 00:03:00,6951.75,6953.50,6951.00,6953.00,414,6952.250,140
2018-02-02 00:04:00,6952.75,6956.50,6952.75,6956.50,304,6954.275,123
...,...,...,...,...,...,...,...
2019-09-04 18:27:00,7695.50,7696.00,7694.75,7695.00,369,7695.475,178
2019-09-04 18:28:00,7695.00,7695.50,7694.75,7695.00,173,7695.025,72
2019-09-04 18:29:00,7695.00,7695.75,7694.50,7695.00,195,7695.125,87


In [4]:
#periods = 20
periods = [5, 10, 20, 40, 80] #, 160, 320]
ema_fast = 120 # number of periods for moving average filter
sl_atr = 1 # stop loss in ATRs
atr_periods = 80 # number of periods to calculate ATR on
time_int = 45 # interval in minutes to be used to define volume candle
commission = 2.1 # level of commision per transaction in dollars
bankroll = 15000

In [5]:
avg_vol = contract.loc['20190621':].volume.rolling(time_int).sum().mean()
avg_vol

14875.34808186604

In [6]:
#avg_vol = 14913

In [7]:
vol_candles = group_by_volume(contract, avg_vol)
vol_candles['atr'] = get_ATR(vol_candles, atr_periods)

In [8]:
vol_candles

,date,open,high,low,close,barCount,volume,atr
label,,,,,,,,
0,2018-02-02 02:42:00,6954.00,6963.50,6912.50,6915.00,7971,14857,51.000000
1,2018-02-02 07:58:00,6915.25,6944.00,6915.00,6930.50,8604,14945,39.862500
2,2018-02-02 10:20:00,6930.50,6939.75,6888.00,6889.50,8018,14473,43.924470
3,2018-02-02 12:04:00,6889.75,6889.75,6857.75,6867.25,9138,16018,40.830641
4,2018-02-02 13:34:00,6867.00,6872.50,6844.50,6866.25,8830,15101,38.134623
...,...,...,...,...,...,...,...,...
12059,2019-09-04 16:37:00,7673.50,7683.75,7668.00,7679.25,7754,15086,24.562230
12060,2019-09-04 17:01:00,7679.25,7686.00,7673.00,7684.50,7864,15298,24.276743
12061,2019-09-04 17:28:00,7684.50,7691.00,7679.50,7688.50,7186,14789,23.961268


In [9]:
data = vol_candles.copy()
data['ema'] = data.close.ewm(span=ema_fast).mean()
#data['min'] = data.close.rolling(periods).min()
#data['max'] = data.close.rolling(periods).max()
rows_to_drop = max(*periods, ema_fast)
#rows_to_drop = max(periods, ema_fast)
data = data.iloc[rows_to_drop:]
data.reset_index(drop=True, inplace=True)

In [10]:
#data['buy_signal'] = (data.close >= data['max']) * 1
#data['sell_signal'] = (data.close <= data['min']) * -1
#data['signal'] = data.buy_signal + data.sell_signal
#del data['buy_signal']
#del data['sell_signal']
data['signal'] = get_signals(data.close, periods)
data['filter'] = np.sign(data['close'] - data['ema'])
data['filtered_signal'] = data['signal'] * ((data['signal'] * data['filter']) == 1)

In [11]:
# while in position maintain open price and transaction direction
data['positions'] = 0
# flag to execute transaction at next data point
data['mark'] = False
# note the reason for transaction at next data point
data['reason'] = ''
# record commission paid
data['commission'] = 0
# record transaction price
data['price'] = 0
# entry price for stop loss calculation
data['entry'] = 0

for item in data.itertuples(): 
    # first row doesn't have to check if we have positions or execute transactions
    if not item.Index == 0:
        # starting position is the same as previous day position
        data.loc[item.Index, 'positions'] = data.loc[(item.Index - 1), 'positions'] 
        data.loc[item.Index, 'entry'] = data.loc[(item.Index - 1), 'entry'] 
        # execute transactions
        if data.loc[(item.Index -  1), 'mark']:
            # close position
            if data.loc[item.Index, 'positions']:
                data.loc[item.Index, 'positions'] = 0
                data.loc[item.Index, 'entry'] = 0
                # record transaction price
                data.loc[item.Index, 'price'] = item.open * np.sign(data.loc[(item.Index - 1), 'entry']) * -1
            # open position
            else:
                data.loc[item.Index, 'positions'] = data.loc[(item.Index - 1), 'signal']
                data.loc[item.Index, 'entry'] = item.open * data.loc[(item.Index - 1), 'signal']
                # record transaction price
                data.loc[item.Index, 'price'] = item.open * data.loc[(item.Index -1), 'signal']
            # record commission paid
            data.loc[item.Index, 'commission'] = commission

    # check for close signal
    if data.loc[item.Index, 'positions'] != 0 and np.sign(item.signal) != 0:
        if np.sign(data.loc[item.Index, 'positions']) != np.sign(item.signal):
            data.loc[item.Index, 'mark'] = True
            data.loc[item.Index, 'reason'] = 'close'
    # check for stop-loss signal
    # long positions
    if data.loc[item.Index, 'positions'] > 0:
        if item.close <= (data.loc[item.Index, 'entry'] - (item.atr * sl_atr)):
            data.loc[item.Index, 'mark'] = True
            data.loc[item.Index, 'reason'] = 'stop-out'
    # short positions
    if data.loc[item.Index, 'positions'] < 0:
        if item.close >= abs((data.loc[item.Index, 'entry'] - (item.atr * sl_atr))):
            data.loc[item.Index, 'mark'] = True
            data.loc[item.Index, 'reason'] = 'stop-out'
    # check for entry signal
    if data.loc[item.Index, 'positions'] == 0:
        if item.filtered_signal != 0:
            data.loc[item.Index, 'mark'] = True
            data.loc[item.Index, 'reason'] = 'entry'
# close any open positions
if data[data.price != 0].price.count() % 2 != 0:
    data.loc[data.index[-1]+1, 'price'] = data.open.iloc[-1] * np.sign(data.entry.iloc[-1]) * -1
    data.loc[data.index[-2]+1, 'entry'] = 0
data.set_index('date', inplace=True)

In [36]:
data.iloc[150:].head(25)

,open,high,low,close,barCount,volume,atr,ema,signal,filter,filtered_signal,positions,mark,reason,commission,price,entry
date,,,,,,,,,,,,,,,,,
2018-02-09 21:18:00,6308.00,6343.50,6281.25,6337.50,8461.0,16733.0,47.682624,6417.441355,0.0,-1.0,0.0,0.0,False,,0.0,0.00,0.00
2018-02-09 21:32:00,6337.75,6373.75,6330.50,6371.75,8000.0,15638.0,47.573055,6416.677923,1.0,-1.0,0.0,0.0,False,,0.0,0.00,0.00
2018-02-09 21:42:00,6371.00,6398.25,6362.75,6398.25,7620.0,14656.0,47.274631,6416.370077,1.0,-1.0,0.0,0.0,False,,0.0,0.00,0.00
2018-02-09 21:48:00,6398.25,6441.50,6393.50,6437.25,7192.0,15756.0,47.292560,6416.718824,1.0,1.0,1.0,0.0,True,entry,0.0,0.00,0.00
2018-02-09 21:54:00,6436.25,6455.50,6417.50,6431.50,7423.0,16579.0,47.062877,6416.965663,0.0,1.0,0.0,1.0,False,,2.1,6436.25,6436.25
2018-02-09 21:58:00,6431.50,6435.25,6398.50,6399.50,5606.0,13562.0,46.807981,6416.674044,0.0,-1.0,0.0,1.0,False,,0.0,0.00,6436.25
2018-02-09 22:13:00,6399.25,6428.25,6389.00,6425.00,10523.0,23490.0,46.621181,6416.813037,0.0,1.0,0.0,1.0,False,,0.0,0.00,6436.25
2018-02-12 01:40:00,6426.00,6447.75,6412.75,6437.75,10524.0,15819.0,46.333963,6417.162499,1.0,1.0,1.0,1.0,False,,0.0,0.00,6436.25
2018-02-12 04:33:00,6437.75,6454.25,6433.50,6440.75,10506.0,14989.0,45.701669,6417.556139,1.0,1.0,1.0,1.0,False,,0.0,0.00,6436.25


In [12]:
pnl = data[['open','positions', 'commission', 'mark', 'reason', 'price', 'entry']]
pnl['mark'] = pnl['mark'].shift(1)
pnl['reason'] = pnl['reason'].shift(1)

In [13]:
pnl

,open,positions,commission,mark,reason,price,entry
date,,,,,,,
2018-02-06 15:51:00,6525.00,0.0,0.0,NaN,NaN,0.00,0.00
2018-02-06 15:55:00,6552.75,0.0,0.0,False,,0.00,0.00
2018-02-06 16:00:00,6594.75,0.0,0.0,False,,0.00,0.00
2018-02-06 16:07:00,6565.75,0.0,0.0,False,,0.00,0.00
2018-02-06 16:15:00,6571.75,0.0,0.0,False,,0.00,0.00
...,...,...,...,...,...,...,...
2019-09-04 17:01:00,7679.25,1.0,0.0,False,,0.00,7656.75
2019-09-04 17:28:00,7684.50,1.0,0.0,False,,0.00,7656.75
2019-09-04 17:54:00,7688.75,1.0,0.0,False,,0.00,7656.75


In [14]:
-pnl.price.sum() * multiplier

36190.0

In [15]:
pnl = pnl[pnl.price != 0]
pnl['close'] = pnl.price.shift(-1)
pnl = pnl[pnl.reason == 'entry']
pnl['pnl'] = -(pnl.entry + pnl.close) * multiplier
pnl['net_pnl'] = pnl.pnl - 2 * pnl.commission
pnl['cum_pnl'] = pnl.pnl.cumsum()
pnl['net_cum_pnl'] = pnl.net_pnl.cumsum()
pnl['balance'] = bankroll + pnl.net_cum_pnl
pnl

,open,positions,commission,mark,reason,price,entry,close,pnl,net_pnl,cum_pnl,net_cum_pnl,balance
date,,,,,,,,,,,,,
2018-02-06 21:07:00,6603.25,1.0,2.1,True,entry,6603.25,6603.25,-6628.50,505.0,500.8,505.0,500.8,15500.8
2018-02-07 19:21:00,6610.25,-1.0,2.1,True,entry,-6610.25,-6610.25,6652.00,-835.0,-839.2,-330.0,-338.4,14661.6
2018-02-07 22:59:00,6579.75,-1.0,2.1,True,entry,-6579.75,-6579.75,6364.75,4300.0,4295.8,3970.0,3957.4,18957.4
2018-02-09 21:54:00,6436.25,1.0,2.1,True,entry,6436.25,6436.25,-6463.75,550.0,545.8,4520.0,4503.2,19503.2
2018-02-14 16:51:00,6610.25,1.0,2.1,True,entry,6610.25,6610.25,-6778.75,3370.0,3365.8,7890.0,7869.0,22869.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-02 04:49:00,7630.25,-1.0,2.1,True,entry,-7630.25,-7630.25,7669.25,-780.0,-784.2,37395.0,35765.4,50765.4
2019-09-03 00:01:00,7608.00,-1.0,2.1,True,entry,-7608.00,-7608.00,7648.75,-815.0,-819.2,36580.0,34946.2,49946.2
2019-09-03 11:09:00,7602.50,-1.0,2.1,True,entry,-7602.50,-7602.50,7632.00,-590.0,-594.2,35990.0,34352.0,49352.0


In [16]:
pnl.pnl.sum()

36190.0

In [17]:
pnl.net_pnl.sum()

34543.60000000002

In [18]:
# percentage of winning transactions
win_pct = round(pnl[pnl.pnl > 0].pnl.count() / pnl.pnl.count(), 4)
win_pct

0.301

In [19]:
# avg. win,  avg. lose, ratio
avg_win = round(pnl[pnl.pnl > 0].pnl.mean(), 2)
avg_lose = round(pnl[pnl.pnl < 0].pnl.mean(), 2)
ratio = round(abs(avg_win / avg_lose), 2)
avg_win, avg_lose, ratio

(1758.05, -625.04, 2.81)

In [20]:
# transaction EV
ev = win_pct * avg_win + (1 - win_pct) * avg_lose
ev

92.27008999999998

In [21]:
# number of transactions
pnl.pnl.count()

392

In [22]:
pnl.pnl.count() * ev

36169.87527999999

In [23]:
pnl.pnl.min(), pnl.pnl.max(), round(abs(pnl.pnl.max() / pnl.pnl.min()), 1)

(-1700.0, 6185.0, 3.6)

In [24]:
daily = pnl.resample('D').last()
daily['return'] = daily.balance.pct_change()

In [25]:
data.commission.sum()

1644.3

In [26]:
perf_stats(daily['return'])

Annual return          0.662559
Cumulative returns     2.196196
Annual volatility      0.527802
Sharpe ratio           1.216455
Calmar ratio           1.976650
Stability              0.715683
Max drawdown          -0.335193
Omega ratio            1.384419
Sortino ratio          2.555838
Skew                        NaN
Kurtosis                    NaN
Tail ratio             2.010906
Daily value at risk   -0.063949
dtype: float64

In [27]:
data.price.count()

11945

In [28]:
data[data.price != 0].price.count()

784